In [1]:
# Add autoreload using % snytax
import IPython
ip = IPython.get_ipython()
if ip is not None:
    ip.run_line_magic('load_ext', 'autoreload')
    ip.run_line_magic('autoreload', '2')

In [2]:
from matplotlib import pyplot as plt
from whar_datasets.adapters.pytorch_location_adapter import PyTorchLocationAdapter
from whar_datasets.adapters.numpy_location import NumpyLocationAdapter
from whar_datasets.support.getter import WHARDatasetID, get_whar_cfg
import rich

In [3]:
def get_dataset(id: WHARDatasetID):
    cfg = get_whar_cfg(id)
    cfg.in_memory = True
    dataset = NumpyLocationAdapter(cfg)
    rich.print(dataset.samples.shape)
    rich.print(dataset.labels.shape)
    return dataset

dataset = get_dataset(WHARDatasetID.DSADS)

Creating config hash...
Checking download...
Download exists.
Checking sessions...
Sessions exist.
Validating common format...
[########################################] | 100% Completed | 1.07 sms
Common format validated.
Checking windowing...
Loading config hash...
Windowing exists.
subject_ids: [0 1 2 3 4 5 6 7]
activity_ids: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
Postprocessing...
Loading all windows from single parquet file...


Splitting windows: 100%|██████████| 17936/17936 [00:02<00:00, 6144.89it/s]


Getting normalization parameters...
Loading config hash...
Creating normalization parameters hash...
Loading normalization parameters hash...
Loading all samples from single parquet file...


Reconstructing samples: 100%|██████████| 17936/17936 [00:02<00:00, 7900.52it/s]


(89680, 1, 125, 9)

(89680,)

In [6]:
dataset.participants

array([0, 0, 0, ..., 7, 7, 7])

In [5]:
train_dataloader, val_dataloader, test_dataloader = dataset.get_dataloaders(16)

AttributeError: 'NumpyLocationAdapter' object has no attribute 'get_dataloaders'

In [ ]:
item = next(iter(train_dataloader))
rich.print(item[0].shape)
rich.print(item[1].shape)
rich.print(item[2].shape)
rich.print(item[3])

torch.Size([16])

torch.Size([16, 128, 9])

torch.Size([16])

[
    tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
    tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
    tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
    tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
    tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
    tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]),
    tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
]

In [ ]:
from whar_datasets.support.dataset_activities import build_activity_alignment
from whar_datasets.support.getter import WHARDatasetID
import numpy as np

pamap_cfg = get_whar_cfg(WHARDatasetID.PAMAP2, datasets_dir="./datasets")  # adjust path if needed
pamap_adapter = NumpyLocationAdapter(pamap_cfg)

dsads_cfg = get_whar_cfg(WHARDatasetID.DSADS, datasets_dir="./datasets")  # adjust path if needed
dsads_adapter = NumpyLocationAdapter(dsads_cfg)

# 1) Build canonical activities and per-dataset mappings
canonical, mappings = build_activity_alignment(
    [WHARDatasetID.PAMAP2, WHARDatasetID.DSADS],
    strategy="intersection",  # or "union", or pass target_classes=[...]
)
print("Canonical:", canonical)

# 2) Remap labels from each dataset
def remap(labels: np.ndarray, mapping: list[int]) -> tuple[np.ndarray, np.ndarray]:
    mapping = np.array(mapping)
    new = mapping[labels]
    keep = new >= 0
    return new[keep], keep  # keep mask drops activities not in canonical

pamap_mapping = mappings[WHARDatasetID.PAMAP2]
dsads_mapping = mappings[WHARDatasetID.DSADS]

pamap_labels_remapped, pamap_keep = remap(pamap_adapter.labels, pamap_mapping)
dsads_labels_remapped, dsads_keep = remap(dsads_adapter.labels, dsads_mapping)

# 3) Filter samples with the keep masks, then concatenate as needed
pamap_samples = pamap_adapter.samples[pamap_keep]
dsads_samples = dsads_adapter.samples[dsads_keep]

Canonical: ['laying', 'sitting', 'standing', 'walking', 'walking_upstairs', 'walking_downstairs']


NameError: name 'pamap_adapter' is not defined